In [11]:
import os
#––– Environment setup –––
os.environ["CUDA_VISIBLE_DEVICES"] = "3"
os.environ["OPENBLAS_NUM_THREADS"] = "1"

In [2]:
import random
from collections import Counter
from itertools import combinations

import torch
import torch.nn.functional as F
import numpy as np
import networkx as nx
from torch_geometric.nn import GAE, GCNConv
from torch_geometric.utils import from_networkx
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm



#––– Community‐comparison utilities –––
def pairwise_community_pairs(communities):
    pairs = set()
    for comm in communities:
        for u, v in combinations(sorted(comm), 2):
            pairs.add((u, v))
    return pairs

def compute_fsame(c1, c2):
    n = sum(len(c) for c in c1)
    M = [[len(set(a).intersection(b)) for b in c2] for a in c1]
    max_row = sum(max(row) for row in M)
    max_col = sum(max(col) for col in zip(*M))
    return 0.5 * (max_row + max_col) / n

def jaccard_index(c1, c2):
    P1 = pairwise_community_pairs(c1)
    P2 = pairwise_community_pairs(c2)
    a = len(P1 & P2)
    b = len(P1 - P2)
    c = len(P2 - P1)
    return 1.0 if (a + b + c) == 0 else a / (a + b + c)

#––– GAE Encoder –––
class Encoder(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def forward(self, x, edge_index):
        x = F.relu(self.conv1(x, edge_index))
        return self.conv2(x, edge_index)

#––– Extract embeddings –––
def get_embeddings(data, hidden_dim=64, emb_dim=16, epochs=200, lr=0.01):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = GAE(Encoder(data.num_features, hidden_dim, emb_dim)).to(device)
    data = data.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    model.train()
    for epoch in tqdm(range(epochs), desc="GAE training"):
        optimizer.zero_grad()
        z = model.encode(data.x, data.edge_index)
        loss = model.recon_loss(z, data.edge_index)
        loss.backward()
        optimizer.step()
    
    model.eval()
    with torch.no_grad():
        z = model.encode(data.x, data.edge_index)
    return z.cpu().numpy()

#––– Evaluation & Visualization –––
from networkx.algorithms.community import modularity

def evaluate_modularity(G, communities, label):
    Q = modularity(G, communities)
    print(f"{label} modularity: {Q:.4f}")
    return Q

def draw_communities(edges, communities, title, fname):
    G = nx.Graph()
    G.add_edges_from(edges)
    pos = nx.spring_layout(G, seed=42)
    node_colors = {node: cid for cid, comm in enumerate(communities) for node in comm}
    plt.figure(figsize=(8, 6))
    nx.draw_networkx_edges(G, pos, alpha=0.3)
    nx.draw_networkx_nodes(
        G, pos,
        node_color=[node_colors[n] for n in G.nodes()],
        cmap='tab20',
        node_size=150,
        alpha=0.9
    )
    plt.title(title)
    plt.axis('off')
    plt.tight_layout()
    plt.savefig(fname)
    plt.close()

def plot_community_sizes(communities, title, fname, loglog=False):
    sizes = sorted([len(c) for c in communities], reverse=True)
    plt.figure(figsize=(6, 4))
    if loglog:
        plt.loglog(sizes, marker='o')
    else:
        plt.bar(range(len(sizes)), sizes)
    plt.title(title)
    plt.xlabel("Community Rank")
    plt.ylabel("Size")
    plt.tight_layout()
    plt.savefig(fname)
    plt.close()

#––– Seeded Label Propagation –––
def seeded_label_propagation(G, init_labels, iterations=10, seed=42):
    random.seed(seed)
    labels = init_labels.copy()
    nodes = list(G.nodes())
    for it in tqdm(range(iterations), desc="Seeded LPA iterations"):
        random.shuffle(nodes)
        for u in nodes:
            nbr_labels = [labels[v] for v in G.neighbors(u)]
            if not nbr_labels:
                continue
            counts = Counter(nbr_labels)
            max_count = max(counts.values())
            choices = [lab for lab, c in counts.items() if c == max_count]
            labels[u] = random.choice(choices)
    communities = []
    for lab in set(labels.values()):
        communities.append([n for n, l in labels.items() if l == lab])
    return communities

#––– Main pipeline –––
def main():
    # 1) Load dataset
    dataset = "zachary"  # or "www" or "zachary"
    if dataset == "coauthorship":
        G_nx = nx.read_edgelist("./data/CA-CondMat.txt", nodetype=int)
    elif dataset == "www":
        G_nx = nx.read_edgelist("./data/web-NotreDame.txt", nodetype=int)
    elif dataset == "zachary":
        G_nx = nx.read_gml("./data/karate.gml", label="id")
    else:
        raise ValueError("Invalid dataset")
    G_nx = nx.convert_node_labels_to_integers(G_nx)
    edges = list(G_nx.edges())

    # 2) Build PyG data
    data = from_networkx(G_nx)
    data.x = torch.eye(G_nx.number_of_nodes(), dtype=torch.float)

    # 3) Get embeddings
    emb = get_embeddings(data, epochs=200)

    # 4) Choose K by silhouette
    best_k, best_score = 3, -1
    for k in tqdm(range(2, 7), desc="Selecting K"):
        labels = KMeans(n_clusters=k, random_state=42).fit_predict(emb)
        score = silhouette_score(emb, labels)
        if score > best_score:
            best_k, best_score = k, score
    print(f"Selected K = {best_k} (silhouette={best_score:.3f})")

    # 5) GAE+KMeans communities
    labels = KMeans(n_clusters=best_k, random_state=42).fit_predict(emb)
    km_comms = [[i for i, l in enumerate(labels) if l == c] for c in range(best_k)]

    # 6) Evaluate & visualize GAE+KMeans
    Q_km = evaluate_modularity(G_nx, km_comms, "GAE+KMeans")
    draw_communities(edges, km_comms, "GAE+KMeans Communities", "d_km.png")
    plot_community_sizes(km_comms, "GAE+KMeans Community Sizes", "c_km.png", loglog=True)
    print('..')
    # 7) Seeded LPA refinement
    init_labels = {i: int(l) for i, l in tqdm(enumerate(labels))}
    lpa_comms = seeded_label_propagation(G_nx, init_labels, iterations=5, seed=42)

    # 8) Evaluate & visualize Seeded LPA
    Q_lpa = evaluate_modularity(G_nx, lpa_comms, "Seeded LPA")
    draw_communities(edges, lpa_comms, "Seeded LPA Communities", "d_lpa.png")
    plot_community_sizes(lpa_comms, "Seeded LPA Community Sizes", "c_lpa.png", loglog=True)
    print('..')
    # 9) Compare partitions
    fs = compute_fsame(km_comms, lpa_comms)
    ji = jaccard_index(km_comms, lpa_comms)
    print(f"fsame between KMeans & LPA: {fs:.4f}")
    print(f"Jaccard between KMeans & LPA: {ji:.4f}")

if __name__ == "__main__":
    main()


Selecting K: 100%|██████████| 5/5 [00:00<00:00, 23.22it/s]


Selected K = 5 (silhouette=0.439)
GAE+KMeans modularity: 0.2060
..


34it [00:00, ?it/s]
Seeded LPA iterations: 100%|██████████| 5/5 [00:00<?, ?it/s]

Seeded LPA modularity: 0.4020


..
fsame between KMeans & LPA: 0.8088
Jaccard between KMeans & LPA: 0.4339


In [5]:
import os

#––– Environment setup & global seed –––
SEED = 50
os.environ["PYTHONHASHSEED"] = str(SEED)
os.environ["CUDA_VISIBLE_DEVICES"] = "3"
os.environ["OPENBLAS_NUM_THREADS"] = "1"

import random
import numpy as np
import torch
from collections import Counter
from itertools import combinations
import networkx as nx
import matplotlib.pyplot as plt
from torch_geometric.nn import GAE, GCNConv
from torch_geometric.utils import from_networkx
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import seaborn as sns
from networkx.algorithms.community import modularity

#––– Reproducibility & performance –––
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = False
torch.backends.cudnn.benchmark = True

#––– Metrics –––
def pairwise_community_pairs(communities):
    pairs = set()
    for comm in communities:
        for u, v in combinations(sorted(comm), 2):
            pairs.add((u, v))
    return pairs

def compute_fsame(c1, c2):
    n = sum(len(c) for c in c1)
    M = [[len(set(a).intersection(b)) for b in c2] for a in c1]
    max_row = sum(max(row) for row in M)
    max_col = sum(max(col) for col in zip(*M))
    return 0.5 * (max_row + max_col) / n

def jaccard_index(c1, c2):
    P1 = pairwise_community_pairs(c1)
    P2 = pairwise_community_pairs(c2)
    a = len(P1 & P2)
    b = len(P1 - P2)
    c = len(P2 - P1)
    return 1.0 if (a + b + c) == 0 else a / (a + b + c)

#––– GAE Encoder –––
class Encoder(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def forward(self, x, edge_index):
        x = torch.relu(self.conv1(x, edge_index))
        return self.conv2(x, edge_index)

#––– Extract embeddings –––
def get_embeddings(data, hidden_dim=64, emb_dim=16, epochs=100, lr=0.01):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = GAE(Encoder(data.num_features, hidden_dim, emb_dim)).to(device)
    data = data.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    model.train()
    for epoch in range(epochs):
        optimizer.zero_grad()
        z = model.encode(data.x, data.edge_index)
        loss = model.recon_loss(z, data.edge_index)
        loss.backward()
        optimizer.step()
    
    model.eval()
    with torch.no_grad():
        z = model.encode(data.x, data.edge_index)
    return z.cpu().numpy()

#––– Plot helpers –––
def plot_community_sizes(communities, title, fname, loglog=False):
    sizes = sorted([len(c) for c in communities], reverse=True)
    plt.figure(figsize=(6, 4))
    if loglog:
        plt.loglog(sizes, marker='o')
    else:
        plt.bar(range(len(sizes)), sizes)
    plt.title(title)
    plt.xlabel("Community Rank")
    plt.ylabel("Size")
    plt.tight_layout()
    plt.savefig(fname, dpi=300)
    plt.close()

def plot_heatmap(matrix, title, fname):
    plt.figure(figsize=(6, 5))
    sns.heatmap(matrix, annot=True, fmt=".2f", cmap="Blues", square=True)
    plt.title(title)
    plt.xlabel("Run")
    plt.ylabel("Run")
    plt.tight_layout()
    plt.savefig(fname, dpi=300)
    plt.close()

#––– Seeded Label Propagation –––
def seeded_label_propagation(G, init_labels, iterations=10, seed=SEED):
    random.seed(seed)
    labels = init_labels.copy()
    nodes = list(G.nodes())
    for _ in range(iterations):
        random.shuffle(nodes)
        for u in nodes:
            nbrs = [labels[v] for v in G.neighbors(u)]
            if not nbrs: continue
            counts = Counter(nbrs)
            max_c = max(counts.values())
            choices = [lab for lab, cnt in counts.items() if cnt == max_c]
            labels[u] = random.choice(choices)
    communities = [[n for n, l in labels.items() if l == lab] for lab in set(labels.values())]
    return communities

#––– Main pipeline –––
def main():
    # Load dataset
    dataset = "zachary"
    if dataset == "coauthorship":
        G_nx = nx.read_edgelist("../data/CA-CondMat.txt", nodetype=int)
    elif dataset == "www":
        G_nx = nx.read_edgelist("../data/web-NotreDame.txt", nodetype=int)
    elif dataset == "zachary":
        G_nx = nx.read_gml("./data/karate.gml", label="id")
    else:
        raise ValueError("Invalid dataset")
    G_nx = nx.convert_node_labels_to_integers(G_nx)

    # Build PyG data
    data = from_networkx(G_nx)
    data.x = torch.eye(G_nx.number_of_nodes(), dtype=torch.float)

    # Get embeddings
    emb = get_embeddings(data, epochs=200)

    # 1. Find optimal number of clusters via silhouette
    scores = {}
    for k in range(2, 4):
        km = KMeans(n_clusters=k, random_state=SEED).fit(emb)
        scores[k] = silhouette_score(emb, km.labels_)
    best_k = max(scores, key=scores.get)
    print(f"Optimal number of clusters by silhouette: {best_k}")

    # 2. Run KMeans 5 times and collect communities
    runs = 5
    km_communities = []
    for i in range(runs):
        labels = KMeans(n_clusters=best_k, random_state=SEED + i).fit_predict(emb)
        comms = [[n for n, l in enumerate(labels) if l == c] for c in range(best_k)]
        km_communities.append(comms)
        if i == 0:
            plot_community_sizes(comms, f"Run {i+1} Community Sizes", f"community_sizes_run{i+1}_zachary.png", loglog=True)

    # Build confusion matrices for Jaccard and fsame
    jaccard_mat = np.zeros((runs, runs))
    fsame_mat = np.zeros((runs, runs))
    for i in range(runs):
        for j in range(runs):
            jaccard_mat[i, j] = jaccard_index(km_communities[i], km_communities[j])
            fsame_mat[i, j]   = compute_fsame(km_communities[i], km_communities[j])

    # Plot and save heatmaps
    plot_heatmap(jaccard_mat, "Jaccard Similarity Across KMeans Runs", "jaccard_confusion_zachary.png")
    plot_heatmap(fsame_mat,   "fsame Similarity Across KMeans Runs",   "fsame_confusion_zachary.png")

    # Print modularity for one example
    Q_example = modularity(G_nx, km_communities[0])
    print(f"Modularity of first run: {Q_example:.4f}")

if __name__ == "__main__":
    main()


Optimal number of clusters by silhouette: 3
Modularity of first run: 0.3991
